# Homework 12 Groups

## Task 1
Задание 1**
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [3]:
%%time
import pandas as pd

ratings = pd.read_csv('ratings.csv', encoding='utf-8')


def class_me(row):
    if row['rating'] <= 2:
        return 'Low'
    elif row['rating'] <= 4:
        return 'Middle'
    elif row['rating'] == 4.5 or row['rating'] == 5:
        return 'Top'

    return 'Undefined'

ratings['class'] = ratings.apply(class_me, axis=1)

Wall time: 12.5 s


## Task 2
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [35]:
%%time
import pandas as pd

geo_data = {
    'Центр': ['москва', 'тула', 'ярославль'],
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

keywords = pd.read_csv('keywords.csv', encoding='utf-8')


### fast way via .loc
for region in geo_data:
    keywords.loc[keywords['keyword'].str.contains( ('|').join(geo_data[region]) ), 'region'] = region
    
keywords.loc[keywords['region'].isnull(), 'region'] = 'undefined'


### slow way via apply function
#def geo_label(row):
#    for region in geo_data:
#        if any(item in row['keyword'] for item in set(geo_data[region])):
#            return region
#    return 'undefined'

#keywords['region'] = keywords.apply(geo_label, axis=1)


Wall time: 1.3 s


## Task 3
Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

для каждой строки пройдите по всем годам списка years
если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [40]:
%%time
import pandas as pd

# ratings.csv and movies.csv from ml-latest-small
ratings = pd.read_csv('ratings.csv', encoding='utf-8')
movies = pd.read_csv('movies.csv', encoding='utf-8')

# extended frame with both rating and title
pool = pd.DataFrame(ratings.merge(movies, on='movieId'))

years = set(map(str, list(range(1950,2011))))

### slow way via apply
def production_year(row):
    for i in years:
        if i in row['title']:
            return i

    return '1900'

pool['year'] = pool.apply(production_year, axis=1)
pool.groupby('year').agg({'rating': 'mean'}).sort_values('rating', ascending=False)


### fast way via .loc
#for year in years:
#    pool.loc[pool['title'].str.contains(year), 'year'] = year
    
#pool.loc[pool['year'].isnull(), 'year'] = '1900'
#pool.groupby('year').agg({'rating': 'mean'}).sort_values('rating', ascending=False)

Wall time: 1.9 s


,rating
year,
1987,4.175676
1971,4.021739
1991,3.986111
1974,3.935484
1959,3.891304
1978,3.864583
1994,3.756728
1982,3.725806
1993,3.722767
